# Intraday Implied Volatility

In [1]:
import pandas as pd
from notebooks.helper_functions import *

%matplotlib inline

/home/jaewonchoi/anaconda3/envs/RLTrader/lib/python3.8/site-packages/py_lets_be_rational/numba_helper.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  return jit(*jit_args, **jit_kwargs)(fun)


## Downloading py_vollib

Implied volatilty calculation does not have a closed form, hence we need to use an iterative approach. Note that vega (or derivative of black scholes price formula with respect to volatiltiy $\sigma$) is an increasing function with respect to volatilty (i.e. $\sigma$). Therefore, for a given call/put option price, there exists a unique implied volatility. Using this observation, we can use iterative methods such as Newton-Raphson method to find the implied volatility. 

However, Newton-Raphson method does not always converge and can give erroneous result for deep in the money options. Many academic research papers tried tackling this problem, the most notable being "Let's be rational" by Peter Jackel. Here we use Jackel's method to compute implied volatilty. Fortunately, there is a python implementation of Jacke's paper which we can install with pip. 
```bash
pip3 install py_vollib
```
and in our python code, we can import it:
```py
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility
```
You can find the source code at [here][1] and documentation [here][2]


[1]: https://github.com/vollib/py_vollib
[2]: https://vollib.org/documentation/1.0.3/py-modindex.html


## Parameters and Helper Functions

In [2]:
tz = "Asia/Seoul"
beg_date_pd = pd.to_datetime("2023-05-26").tz_localize(tz).date()
end_date_pd = pd.to_datetime("2023-05-26").tz_localize(tz).date()
beg_time = "09:00:01.0"
end_time = "15:19:59.0"

elw_issue_name = "KOSPI200"
issuer_prefix = "KIS"   # korea investment group 한투
underlying_asset_isin = "KR4101T60006"
r = 0.0375     # continuously compounded risk-free interest rate
q = 0.02       # continuously compounded dividend yield
debug=True

In [3]:
import pickle

def save_to_pickle(data_list, file_path):
    try:
        with open(file_path, 'wb') as file:
            pickle.dump(data_list, file)
        print("Data has been successfully saved to", file_path)
    except Exception as e:
        print("Error occurred while saving data:", e)

def load_from_pickle(file_path):
    try:
        with open(file_path, 'rb') as file:
            data_list = pickle.load(file)
        return data_list
    except Exception as e:
        print("Error occurred while loading data:", e)
        return None


### Fetching data

In [4]:
for date in pd.date_range(beg_date_pd, end_date_pd):
    date_str = date.strftime("%Y-%m-%d")
    print(date_str)

    # check if the date is a trading date, if not continue
    if not is_trading_date(date_str):
        continue

    elw_data = ELWData(
        date_str, elw_issue_name, issuer_prefix, 
        r=r, q=q,
        underlying_asset_isin=underlying_asset_isin, 
        tz=tz, debug=debug)

    print("call", len(elw_data.call_isins), elw_data.call_isins)
    print("put", len(elw_data.put_isins), elw_data.put_isins)

    elw_marketdepth = ELWMarketdepthData(
        date_str,
        elw_data.call_isins + elw_data.put_isins,
        [elw_data.underlying_asset_isin],
        beg_time=beg_time,
        end_time=end_time,
        tz=tz,
        debug=debug
    )
    
    elw_data.vol_scatter_plot(elw_data.call_vol_surface, elw_data.put_vol_surface)
    elw_data.vol_surface_plot(elw_data.call_vol_surface, elw_data.put_vol_surface)

    call_vol_surfaces = []
    put_vol_surfaces = []
    next_ts = pd.Timestamp(f"{date_str}T09:05:00.0", tz=tz)
    for k, row in enumerate(elw_marketdepth):
        
        if row["isin"] == elw_data.underlying_asset_isin:
            elw_data.update_underlying_price(row["isin"], row["weighted_midprice"], row["system_datetime"])
        else:
            # elw got updated
            elw_data.update_elw(row["isin"], row["weighted_midprice"], row["system_datetime"])

        if row["system_datetime"] > next_ts:
            next_ts += pd.Timedelta(minutes=5)
            call_vol_surfaces.append(elw_data.call_vol_surface.copy())
            put_vol_surfaces.append(elw_data.put_vol_surface.copy())

        if k % 25000 == 0:
            print(f"updating volatility at {row['system_datetime']}...")


# Save the data to a pickle file
save_to_pickle(call_vol_surfaces, "call_vol_surfaces.pkl")
save_to_pickle(put_vol_surfaces, "put_vol_surfaces.pkl")



2023-05-26


,isin,base_price,prev_day_closing_price,abbreviated_issue_name_in_en,date
0,KR4101T60006,336.0,336.0,F 202306,2023-05-26


call 255 ['KRA5731EED35', 'KRA5731EDD36', 'KRA5731EGCA6', 'KRA5731EFCA8', 'KRA5731EECA1', 'KRA5731F5CA4', 'KRA5731FZCA3', 'KRA5731GSCA6', 'KRA5731EDCA3', 'KRA5731F4CA7', 'KRA5731FYCA6', 'KRA5731GRCA8', 'KRA5731ECCA5', 'KRA5731F3CA9', 'KRA5731FXCA8', 'KRA5731GQCA0', 'KRA5731EBCA7', 'KRA5731F2CA1', 'KRA5731FWCA0', 'KRA5731GPCA2', 'KRA5731DGD18', 'KRA5731EACA9', 'KRA5731EBD12', 'KRA5731F1CA3', 'KRA5731F6D10', 'KRA5731FVCA2', 'KRA5731GNCA7', 'KRA5731DFD19', 'KRA5731E9CA7', 'KRA5731EAD13', 'KRA5731F0CA5', 'KRA5731F5D11', 'KRA5731FTCA6', 'KRA5731GMCA9', 'KRA5731DED10', 'KRA5731E9D10', 'KRA5731E8CA9', 'KRA5731F4D12', 'KRA5731EZCA6', 'KRA5731FSCA8', 'KRA5731GLCA1', 'KRA5731DDD11', 'KRA5731E7CA1', 'KRA5731E8D11', 'KRA5731EYCA9', 'KRA5731F3D13', 'KRA5731FRCA0', 'KRA5731GKCA3', 'KRA5731DCD12', 'KRA5731E6CA3', 'KRA5731E7D12', 'KRA5731EXCA1', 'KRA5731F2D14', 'KRA5731FQCA2', 'KRA5731GJCA5', 'KRA5731LHC75', 'KRA5731DBD13', 'KRA5731E6D13', 'KRA5731EWCA3', 'KRA5731F1D15', 'KRA5731FPCA4', 'KRA5731GHCA9'

0.0 0.6515134519412222
0.0 0.0
0.3616670187783834 0.6515134519412222


updating volatility at 2023-05-26 09:00:01.045358767+09:00...
updating volatility at 2023-05-26 09:01:39.411148992+09:00...
updating volatility at 2023-05-26 09:03:43.643102216+09:00...
updating volatility at 2023-05-26 09:06:07.981260193+09:00...
updating volatility at 2023-05-26 09:08:19.876255262+09:00...
updating volatility at 2023-05-26 09:11:30.103289996+09:00...
updating volatility at 2023-05-26 09:14:57.690559614+09:00...
updating volatility at 2023-05-26 09:17:35.337466122+09:00...
updating volatility at 2023-05-26 09:21:11.689315257+09:00...
updating volatility at 2023-05-26 09:24:25.026190542+09:00...
updating volatility at 2023-05-26 09:28:19.984093237+09:00...
updating volatility at 2023-05-26 09:31:53.165849965+09:00...
updating volatility at 2023-05-26 09:36:36.249314137+09:00...
updating volatility at 2023-05-26 09:41:06.107326049+09:00...
updating volatility at 2023-05-26 09:46:06.690002544+09:00...
updating volatility at 2023-05-26 09:50:50.302028603+09:00...
updating

In [ ]:
call_vol_surfaces_ld = load_from_pickle("call_vol_surfaces.pkl")
put_vol_surfaces_ld = load_from_pickle("put_vol_surfaces.pkl")

print(call_vol_surfaces_ld[10])

{'KRA5731EED35': {'price': 6985.454545454545, 'exercise_price': 265.0, 'moneyness': 0.7886904761904762, 'maturity': 17, 'sigma': 0}, 'KRA5731EDD36': {'price': 6640.886075949367, 'exercise_price': 270.0, 'moneyness': 0.8035714285714286, 'maturity': 17, 'sigma': 0}, 'KRA5731EGCA6': {'price': 6255.454545454545, 'exercise_price': 272.5, 'moneyness': 0.8110119047619048, 'maturity': 17, 'sigma': 0}, 'KRA5731EFCA8': {'price': 6013.636363636364, 'exercise_price': 275.0, 'moneyness': 0.8184523809523809, 'maturity': 17, 'sigma': 0}, 'KRA5731EECA1': {'price': 5771.818181818182, 'exercise_price': 277.5, 'moneyness': 0.8258928571428571, 'maturity': 17, 'sigma': 0}, 'KRA5731F5CA4': {'price': 5950.0, 'exercise_price': 277.5, 'moneyness': 0.8258928571428571, 'maturity': 52, 'sigma': 0}, 'KRA5731FZCA3': {'price': 6017.5, 'exercise_price': 277.5, 'moneyness': 0.8258928571428571, 'maturity': 80, 'sigma': 0}, 'KRA5731GSCA6': {'price': 6117.5, 'exercise_price': 277.5, 'moneyness': 0.8258928571428571, 'matu

In [ ]:
ELWData.vol_scatter_plot(call_vol_surfaces_ld[12], put_vol_surfaces_ld[12])
ELWData.vol_surface_plot(call_vol_surfaces_ld[12], put_vol_surfaces_ld[12])



0.0 0.18343216467846238
0.0 nan
0.18343216467846238 nan


In [ ]:
ELWData.vol_scatter_plot(call_vol_surfaces_ld[24], put_vol_surfaces_ld[24])
ELWData.vol_surface_plot(call_vol_surfaces_ld[24], put_vol_surfaces_ld[24])


0.0 0.18343216467846238
0.0 nan
0.18343216467846238 nan


In [ ]:
ELWData.vol_scatter_plot(call_vol_surfaces_ld[72], put_vol_surfaces_ld[72])
ELWData.vol_surface_plot(call_vol_surfaces_ld[72], put_vol_surfaces_ld[72])


0.0 0.18343216467846238
0.0 nan
0.18343216467846238 nan
